In [100]:
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense 
from keras.optimizers import Adam
# from keras.callbacks import TensorBoard
import time
# PLOT=True
PLOT=False

SEQ_LEN=60

Using plaidml.keras.backend backend.


In [101]:
url = "./datasets/DAT_ASCII_EURUSD_M1_2017.csv"
df = pd.read_csv(url, names=list(["date","open", "high", "low", "close", "volume"]), header=None, sep=";")
df.drop("volume", 1, inplace=True)
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df.set_index('date', inplace=True)

df.head()

,open,high,low,close
date,,,,
2017-01-02 02:00:00,1.05155,1.05197,1.05155,1.05190
2017-01-02 02:01:00,1.05209,1.05209,1.05177,1.05179
2017-01-02 02:02:00,1.05177,1.05198,1.05177,1.05178
2017-01-02 02:03:00,1.05188,1.05200,1.05188,1.05200
2017-01-02 02:04:00,1.05196,1.05204,1.05196,1.05203


In [102]:
if PLOT:
    df.plot(subplots=True, layout=(2, 2), figsize=(40, 20), sharex=False)

In [103]:
data_set = df.iloc[:, 2:3].values #close
print(data_set)

[[1.05155]
 [1.05177]
 [1.05177]
 ...
 [1.19961]
 [1.19974]
 [1.1998 ]]


In [104]:
print(data_set.shape)

(371635, 1)


In [105]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(data_set)
print(data_set)

[[1.05155]
 [1.05177]
 [1.05177]
 ...
 [1.19961]
 [1.19974]
 [1.1998 ]]


In [106]:
X_train = []
y_train = []
for i in range(SEQ_LEN, training_set_scaled.size-1):
    X_train.append(training_set_scaled[i-SEQ_LEN:i, 0])
    y_train.append(training_set_scaled[i+1, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train[0])

[0.10006866 0.10132738 0.10132738 0.10195675 0.10241446 0.10241446
 0.1029294  0.10315826 0.10310104 0.10304383 0.10327269 0.10338712
 0.10315826 0.10298661 0.10321547 0.10310104 0.10327269 0.10132738
 0.10127017 0.10104131 0.10109852 0.10127017 0.10167067 0.10127017
 0.10167067 0.1013846  0.10149903 0.10132738 0.10155624 0.10132738
 0.1017851  0.10212839 0.10252889 0.10264332 0.10275775 0.10264332
 0.10161346 0.10189953 0.10115574 0.10075524 0.10075524 0.10069802
 0.10086966 0.10092688 0.10041195 0.10069802 0.10058359 0.0998398
 0.09966815 0.09989701 0.0994393  0.09955372 0.09852386 0.09875272
 0.098295   0.0986955  0.09858107 0.09978258 0.09955372 0.09995423]


In [107]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train[0])

[[0.10006866]
 [0.10132738]
 [0.10132738]
 [0.10195675]
 [0.10241446]
 [0.10241446]
 [0.1029294 ]
 [0.10315826]
 [0.10310104]
 [0.10304383]
 [0.10327269]
 [0.10338712]
 [0.10315826]
 [0.10298661]
 [0.10321547]
 [0.10310104]
 [0.10327269]
 [0.10132738]
 [0.10127017]
 [0.10104131]
 [0.10109852]
 [0.10127017]
 [0.10167067]
 [0.10127017]
 [0.10167067]
 [0.1013846 ]
 [0.10149903]
 [0.10132738]
 [0.10155624]
 [0.10132738]
 [0.1017851 ]
 [0.10212839]
 [0.10252889]
 [0.10264332]
 [0.10275775]
 [0.10264332]
 [0.10161346]
 [0.10189953]
 [0.10115574]
 [0.10075524]
 [0.10075524]
 [0.10069802]
 [0.10086966]
 [0.10092688]
 [0.10041195]
 [0.10069802]
 [0.10058359]
 [0.0998398 ]
 [0.09966815]
 [0.09989701]
 [0.0994393 ]
 [0.09955372]
 [0.09852386]
 [0.09875272]
 [0.098295  ]
 [0.0986955 ]
 [0.09858107]
 [0.09978258]
 [0.09955372]
 [0.09995423]]


In [108]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:


model = Sequential()

model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))

opt = Adam(lr=0.001, decay=1e-6)

NAME = f"PRED-{int(time.time())}"  
# tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(optimizer=opt,loss='mean_squared_error')

model.fit(X_train,y_train,epochs=10,batch_size=32,validation_data=[X_test, y_test])

model.save('histdata.h5')